In [110]:
import pandas as pd
import csv as csv
import matplotlib.pyplot as plt

In [111]:
init_traffic_data = pd.read_csv("traffic_accidents.csv")

In [112]:
#dropping unneeded columns
traffic_data = init_traffic_data.drop(labels=["GEO_X","GEO_Y","OBJECTID_1","INCIDENT_ID","OFFENSE_ID","LAST_OCCURENCE_DATE","REPORTED_DATE","OFFENSE_TYPE_ID","OFFENSE_CATEGORY_ID"],axis=1)

In [113]:
#renaming columns a *little* better
traffic_data.rename(columns={"FIRST_OCCURENCE_DATE":"ACCIDENT_DATE","GEO_LON":"LON","GEO_LAT":"LAT"},inplace=True)
traffic_data.head()

,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,ACCIDENT_DATE,INCIDENT_ADDRESS,LON,LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND
0,5401,0,2012-11-02 18:20:00,1900 BLOCK S SHERIDAN BLVD,-105.052849,39.682112,4,421,Harvey Park,0,0
1,5441,0,2012-11-05 14:16:00,E HAMPDEN AVE / S YOSEMITE ST,-104.884786,39.653067,3,323,Hampden South,0,0
2,5401,0,2012-04-02 17:00:00,1300 BLOCK N BROADWAY ST,-104.986886,39.737147,6,623,Capitol Hill,0,0
3,5441,0,2012-06-11 17:05:00,600 BLOCK E EXPOSITION AVE,-104.979833,39.703898,3,311,Washington Park West,0,0
4,5441,0,2012-03-23 11:01:00,I25 HWYSB / W COLFAX AVE,-105.014162,39.740439,1,121,Lincoln Park,0,0


In [ ]:
#splitting date&time stamp to usable columns
date_split = traffic_data["ACCIDENT_DATE"].str.split(" ",expand=True,n=1)
traffic_data["DATE"] = date_split[0]
traffic_data["TIME"] = date_split[1]
date_split = traffic_data["DATE"].str.split("-",expand=True,n=2)
traffic_data["YEAR"] = date_split[0]
traffic_data["MONTH"] = date_split[1]
traffic_data["DAY"] = date_split[2]
traffic_data.drop(labels=["ACCIDENT_DATE","DATE"],axis=1,inplace=True)
traffic_data['OVERALL_OFFENSE_CODE'] = traffic_data['OFFENSE_CODE'].map(str) + traffic_data['OFFENSE_CODE_EXTENSION'].map(str)

#dropping 2019 data as it's not a full year
traffic_data = traffic_data[traffic_data['YEAR'] != "2019"]

traffic_data.head()

,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,INCIDENT_ADDRESS,LON,LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND,TIME,YEAR,MONTH,DAY,OVERALL_OFFENSE_CODE
0,5401,0,1900 BLOCK S SHERIDAN BLVD,-105.052849,39.682112,4,421,Harvey Park,0,0,18:20:00,2012,11,02,54010
1,5441,0,E HAMPDEN AVE / S YOSEMITE ST,-104.884786,39.653067,3,323,Hampden South,0,0,14:16:00,2012,11,05,54410
2,5401,0,1300 BLOCK N BROADWAY ST,-104.986886,39.737147,6,623,Capitol Hill,0,0,17:00:00,2012,04,02,54010
3,5441,0,600 BLOCK E EXPOSITION AVE,-104.979833,39.703898,3,311,Washington Park West,0,0,17:05:00,2012,06,11,54410
4,5441,0,I25 HWYSB / W COLFAX AVE,-105.014162,39.740439,1,121,Lincoln Park,0,0,11:01:00,2012,03,23,54410


In [ ]:
#readding cleaned up offense descriptions, since they're way more useful than a code

descriptionlist = []
for eachcode in traffic_data['OVERALL_OFFENSE_CODE']:
    if eachcode == "54010":
        descriptionlist.append("Hit and Run")
    elif eachcode =="54200":
        descriptionlist.append("DUI")
    elif eachcode =="54410":
        descriptionlist.append("Routine Accident")
    elif eachcode =="54411":
        descriptionlist.append("Serious Bodily Injury")
    elif eachcode =="54412":
        descriptionlist.append("Fatal")
    elif eachcode =="54413":
        descriptionlist.append("Police")

traffic_data['OFFENSE_DESCRIPTION'] = descriptionlist

In [ ]:
#reorder the dataframe
traffic_data = traffic_data[['OVERALL_OFFENSE_CODE','OFFENSE_DESCRIPTION','YEAR','MONTH','DAY','TIME','LAT','LON','INCIDENT_ADDRESS','DISTRICT_ID','PRECINCT_ID','NEIGHBORHOOD_ID','BICYCLE_IND','PEDESTRIAN_IND']]
traffic_data.head()

,OVERALL_OFFENSE_CODE,OFFENSE_DESCRIPTION,YEAR,MONTH,DAY,TIME,LAT,LON,INCIDENT_ADDRESS,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND
0,54010,Hit and Run,2012,11,02,18:20:00,39.682112,-105.052849,1900 BLOCK S SHERIDAN BLVD,4,421,Harvey Park,0,0
1,54410,Routine Accident,2012,11,05,14:16:00,39.653067,-104.884786,E HAMPDEN AVE / S YOSEMITE ST,3,323,Hampden South,0,0
2,54010,Hit and Run,2012,04,02,17:00:00,39.737147,-104.986886,1300 BLOCK N BROADWAY ST,6,623,Capitol Hill,0,0
3,54410,Routine Accident,2012,06,11,17:05:00,39.703898,-104.979833,600 BLOCK E EXPOSITION AVE,3,311,Washington Park West,0,0
4,54410,Routine Accident,2012,03,23,11:01:00,39.740439,-105.014162,I25 HWYSB / W COLFAX AVE,1,121,Lincoln Park,0,0


In [ ]:
#group by year
traffic_data_groupby_year = traffic_data.groupby(by="YEAR")
traffic_data_groupby_year.count()

,OVERALL_OFFENSE_CODE,OFFENSE_DESCRIPTION,MONTH,DAY,TIME,LAT,LON,INCIDENT_ADDRESS,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND
YEAR,,,,,,,,,,,,,
2012,20560,20560,20560,20560,20560,20560,20560,20560,20560,20560,20560,20560,20560
2013,21182,21182,21182,21182,21182,21182,21182,21182,21182,21182,21182,21182,21182
2014,22444,22444,22444,22444,22444,22444,22444,22444,22444,22444,22444,22444,22444
2015,23842,23842,23842,23842,23842,23842,23842,23842,23842,23842,23842,23842,23842
2016,24254,24254,24254,24254,24254,24254,24254,24254,24254,24254,24254,24254,24254
2017,24617,24617,24617,24617,24617,24617,24617,24617,24617,24617,24617,24617,24617
2018,24905,24905,24905,24905,24905,24905,24905,24905,24905,24905,24905,24905,24905


In [ ]:
#group by offense description
traffic_data_groupby_offense = traffic_data.groupby(by=["OFFENSE_DESCRIPTION"])
traffic_data_groupby_offense.count()

,OVERALL_OFFENSE_CODE,YEAR,MONTH,DAY,TIME,LAT,LON,INCIDENT_ADDRESS,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND
OFFENSE_DESCRIPTION,,,,,,,,,,,,,
DUI,5030,5030,5030,5030,5030,5030,5030,5030,5030,5030,5030,5030,5030
Fatal,331,331,331,331,331,331,331,331,331,331,331,331,331
Hit and Run,43112,43112,43112,43112,43112,43112,43112,43112,43112,43112,43112,43112,43112
Police,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570
Routine Accident,109538,109538,109538,109538,109538,109538,109538,109538,109538,109538,109538,109538,109538
Serious Bodily Injury,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223


In [ ]:
#group by offense description THEN year
traffic_data_groupby_offenseandyear = traffic_data.groupby(by=["OFFENSE_DESCRIPTION","YEAR"])
traffic_data_groupby_offenseandyear.count()

In [ ]:
#group by offense year THEN description
traffic_data_groupby_yearandoffense = traffic_data.groupby(by=["YEAR","OFFENSE_DESCRIPTION"])
traffic_data_groupby_yearandoffense.count()